# Introduction to NLP Fundamentals in Tensorflow

Natural Language Process (NLP) is the technology used to aid computers to undestand human's natural language. (could be sequences text or speech)

Another common term for NLP problem is sequence to sequence problems (seq2seq).

In [60]:
# Import libraries
import random
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

print(f'Tensorflow Version: {tf.__version__}')

Tensorflow Version: 2.12.0


## Check for GPU

In [4]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-037266af-d251-3320-bd44-174ca57e46b6)


## Get the helper function and import some functions

In [5]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_histories

## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster)

Kaggle source here for competition: https://www.kaggle.com/competitions/nlp-getting-started

In [6]:
# Download data as zip
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-08-13 14:11:44--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.153.128, 142.250.145.128, 74.125.128.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.153.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  1.27MB/s    in 0.5s    

2023-08-13 14:11:45 (1.27 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [9]:
# Unzip data
zip_ref = zipfile.ZipFile('nlp_getting_started.zip')
zip_ref.extractall()
zip_ref.close()

## Becoming one with the data

We got the data and now, we need to understand our data that's wht lets visualize the text data.

In [15]:
# Read the data and turn it pandas dataframe
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [16]:
# Check out the first 5 row
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [22]:
# Get the info from train dataset
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [24]:
# Check the text row
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [33]:
# Shuffle train data
train_df_shuffled = train_df.sample(frac=1, random_state=42)

In [34]:
# Check the shuffled data
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [35]:
# How does look like test data?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [37]:
# How many examples of each class?
train_df_shuffled['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [39]:
# How many total samples?
print(f'Train total samples: {len(train_df_shuffled)}')
print(f'Test total samples: {len(test_df)}')

Train total samples: 7613
Test total samples: 3263


In [59]:
# Let's visualize the some random training examples
random_samples = random.sample(range(0, len(train_df_shuffled)), 5)

for i in random_samples:
    target = train_df_shuffled['target'][i]
    text = train_df_shuffled['text'][i]
    print(f'Target: {target}', '(Real Diaster)' if target == 1 else '(Not Real Diaster)')
    print(f'Text: {text}', '\n')

Target: 0 (Not Real Diaster)
Text: Wrecked emotions. 

Target: 1 (Real Diaster)
Text: Pic of 16yr old PKK suicide bomber who detonated bomb in Turkey Army trench released http://t.co/gnynJHnE6j http://t.co/1fuNEMes7M 

Target: 1 (Real Diaster)
Text: Californian Bush Fires 2015 http://t.co/rjdX29wosp 

Target: 1 (Real Diaster)
Text: Someone teaching you that obedience will obliterate trials in your life is trying to sell you a used car. Jesus's life blows that theory.' 

Target: 1 (Real Diaster)
Text: IJ: Texas Seeks Comment on Rules for Changes to Windstorm Insurer http://t.co/h132iuL7MU 



### Split data into training and validation sets

In [69]:
# Use train_test_split to split our data
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [70]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [71]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers